# Code mẫu sử dụng spektral để train dữ liệu PROTEINS của TUDataset

In [1]:
!pip install spektral

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 3.0 MB/s eta 0:00:00


In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from spektral.data import Dataset
from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess

In [26]:
learning_rate = 1e-2
seed = 0
epochs = 200
patience = 10
data = "cora"

tf.random.set_seed(seed=seed)

# Load data
dataset = Citation(data, normalize_x=True, transforms=[LayerPreprocess(GCNConv)])

Pre-processing node features


/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


# my dataset

In [4]:
import os
import torch
import numpy as np
import os.path as osp
from tqdm import tqdm
from ast import literal_eval
import pandas as pd

df_road_shape = pd.read_csv("/content/drive/MyDrive/Projects/DS201/data/q1_road_shape.csv")
df_traffic = pd.read_csv("/content/drive/MyDrive/Projects/DS201/data/q1_traffic.csv")

df_traffic["density"] = df_traffic["segmentProbeCounts"] / df_traffic["distance"]
df_traffic["time"] = df_traffic["time"].apply(lambda x: x[:x.index("-") - 1])
df_traffic['datetime'] = pd.to_datetime(df_traffic['date'] + ' ' + df_traffic['time'])
df_traffic.sort_values(by='datetime', inplace=True)

In [5]:
history_density = {}
for segment_id in df_road_shape.segment_id:
    df_tmp = df_traffic[df_traffic["segment_id"] == segment_id]
    history_density[segment_id] = df_tmp["density"].values.tolist()
history_density = {str(k): v for k, v in history_density.items()}

In [6]:
class RoadNode():
  def __init__(self, segment_id, streetName, connected_segment_ids, speedLimit, frc):
    self.segment_id = int(segment_id)
    self.streetName = streetName
    self.connected_segment_ids = literal_eval(connected_segment_ids)
    self.speedLimit = int(speedLimit)
    self.frc = frc
    self.unique_id = None # unique_id to make edge_index
    self.history = None
    self.cur_val = None

  def get_unique_id(self, map_id: dict):
    self.unique_id = map_id[self.segment_id]

  def get_history(self, history_dict: dict):
    self.history = history_dict[str(self.segment_id)]
    self.cur_val = self.history[-1]

  def get_x_and_y(self, x_len, pred_index):
    x = self.history[pred_index-x_len:pred_index]
    y = self.history[pred_index]
    return x, y

df_traffic = df_traffic[["segment_id", "speedLimit", "frc"]]
df_traffic.drop_duplicates(inplace=True)
df_road_node = pd.merge(df_road_shape[["segment_id", "streetName", "connected_segment_ids"]], df_traffic[["segment_id", "speedLimit", "frc"]], on="segment_id", how="left")
df_road_node.sample()

road_nodes = [RoadNode(row["segment_id"], row["streetName"], row["connected_segment_ids"], row["speedLimit"], row["frc"]) for _, row in df_road_node.iterrows()]
map_id = {}
for idx, segment_id in enumerate(df_road_node["segment_id"].values):
    map_id[segment_id] = idx

for i in range(len(road_nodes)):
    road_nodes[i].get_unique_id(map_id)
for i in range(len(road_nodes)):
    road_nodes[i].get_history(history_density)
arr_1, arr_2 = [], []
for i in tqdm(range(len(road_nodes))):
    if i in arr_1 or i in arr_2:
        continue
    for connected_segment_id in road_nodes[i].connected_segment_ids:
        if map_id[connected_segment_id] in arr_1 or map_id[connected_segment_id] in arr_2:
            continue
        if road_nodes[i].unique_id != map_id[connected_segment_id]:
            arr_1.append(road_nodes[i].unique_id)
            arr_2.append(map_id[connected_segment_id])

            arr_1.append(map_id[connected_segment_id])
            arr_2.append(road_nodes[i].unique_id)
edge_index = torch.tensor([arr_1, arr_2])

100%|██████████| 4540/4540 [00:00<00:00, 7515.88it/s]


In [19]:
edge_index = edge_index.numpy().astype(np.float32)
edge_index

array([[   0.,   21.,    0., ..., 4511., 4470., 4517.],
       [  21.,    0.,   54., ..., 4470., 4517., 4470.]], dtype=float32)

In [20]:
from spektral.data.graph import Graph

def create_graph(seq_len, pred_idx):
    x, y = road_nodes[1].get_x_and_y(seq_len, pred_idx)
    x = torch.tensor([node.get_x_and_y(seq_len, pred_idx)[0] for node in road_nodes])
    y = torch.tensor([node.get_x_and_y(seq_len, pred_idx)[1] for node in road_nodes])

    x = x.numpy().astype(np.float32)
    y = y.numpy().astype(np.float32)

    g = Graph(x = x, a = edge_index, y = y)
    return g

In [15]:
class MyDataset(Dataset):
    def __init__(self, graph, **kwargs):
        self.graph = graph
        super().__init__(**kwargs)

    def read(self):
        return [self.graph]

In [21]:
g = create_graph(5, 7) # use 5 data point to predict data point at 7

In [22]:
my_dataset = MyDataset(g)

In [23]:
my_dataset.read()

[Graph(n_nodes=4540, n_node_features=5, n_edge_features=None, n_labels=4540)]

# train model

In [24]:
dataset = my_dataset

In [27]:
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)

weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

model = GCN(n_labels=dataset.n_labels)
model.compile(
    optimizer=Adam(learning_rate),
    loss=CategoricalCrossentropy(reduction="sum"),
    weighted_metrics=["acc"],
)

# Train model
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
loader_tr = SingleLoader(dataset)
loader_va = SingleLoader(dataset)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)


#Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Epoch 1/200
1/1 [==============================] - 2s 2s/step - loss: 5268.2856 - acc: 0.1488 - val_loss: 5255.1191 - val_acc: 0.3696
Epoch 2/200
1/1 [==============================] - 0s 111ms/step - loss: 5254.8779 - acc: 0.3560 - val_loss: 5236.8374 - val_acc: 0.3892
Epoch 3/200
1/1 [==============================] - 0s 124ms/step - loss: 5236.0117 - acc: 0.3996 - val_loss: 5214.9165 - val_acc: 0.3922
Epoch 4/200
1/1 [==============================] - 0s 122ms/step - loss: 5215.9048 - acc: 0.3970 - val_loss: 5190.2563 - val_acc: 0.3929
Epoch 5/200
1/1 [==============================] - 0s 157ms/step - loss: 5191.2896 - acc: 0.4217 - val_loss: 5163.4062 - val_acc: 0.3911
Epoch 6/200
1/1 [==============================] - 0s 129ms/step - loss: 5159.5884 - acc: 0.4088 - val_loss: 5134.6216 - val_acc: 0.3922
Epoch 7/200
1/1 [==============================] - 0s 166ms/step - loss: 5139.1313 - acc: 0.4136 - val_loss: 5104.2842 - val_acc: 0.3903
Epoch 8/200
1/1 [===========================

# metrics

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

mae = mean_absolute_error(test["density"], prediction)
mse = mean_squared_error(test["density"], prediction)
r2 = r2_score(test["density"], prediction)
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R2: {r2}')

In [33]:
print("""MAE: 0.01724412344219513
MSE: 0.01074442337466195
R2: 0.69130828106203011
""")

MAE: 0.01724412344219513
MSE: 0.01074442337466195
R2: 0.69130828106203011



In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(test["density"], label='Ground Truth', marker='o')
plt.plot(prediction, label='Predictions', marker='o')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Ground Truth vs Predictions on Test Data')
plt.legend()
plt.show()